# Model

In [1]:
pip install RegscorePy

In [2]:
pip install tensorflow==1.14

     |████████████████████████████████| 109.2MB 89kB/s 
     |████████████████████████████████| 491kB 43.1MB/s 
     |████████████████████████████████| 3.2MB 32.8MB/s 
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflow 2.2.0rc2
    Uninstalling tensorflow-2.2.0rc2:
      Successfully uninstalled tensorflow-2.2.0rc2


In [3]:
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout, Activation
import matplotlib.pyplot as plt
import RegscorePy 
from RegscorePy import *

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of 

In [4]:
df_f = pd.read_excel('Data.xlsx',sheet_name='Sheet1')
df_hc = pd.read_excel('Data.xlsx',sheet_name='Sheet2')
df_f['Date'] = pd.to_datetime(df_f['Date'])
df_hc['Date'] = pd.to_datetime(df_hc['Date'])
df_f.set_index('Date', inplace=True)
df_hc.set_index('Date', inplace=True)
data_f = df_f[0:1455]
data_hc = df_hc[0:1455]
print(data_f)
print(data_hc)

FileNotFoundError: ignored

In [0]:
fig,(ax1, ax2) = plt.subplots(1, 2, figsize = (16,8))
ax1.plot(data_f)
ax1.set_title('Close Price History DJUSGF')
ax1.set(xlabel='Date',ylabel='Close Price')

ax2.plot(data_hc)
ax2.set_title('Close Price History DJUSHC')
ax2.set(xlabel='Date',ylabel='Close Price')

plt.tight_layout()

In [0]:
dt_f = data_f.filter(['DJUSGF'])
dataset_f = dt_f.values
dt_hc = data_hc.filter(['DJUSHC'])
dataset_hc = dt_hc.values
training_data_len = 1455

In [0]:
scaler_f = MinMaxScaler(feature_range=(0,1))
scaled_data_f = scaler_f.fit_transform(dataset_f)
scaler_hc = MinMaxScaler(feature_range=(0,1))
scaled_data_hc = scaler_hc.fit_transform(dataset_hc)

In [0]:
train_data_f = scaled_data_f[0:training_data_len , :]
x_train_f = []
y_train_f = []

for i in range(60, len(train_data_f)):
  x_train_f.append(train_data_f[i-60:i, 0])
  y_train_f.append(train_data_f[i, 0])

train_data_hc = scaled_data_hc[0:training_data_len , :]
x_train_hc = []
y_train_hc = []

for i in range(60, len(train_data_hc)):
  x_train_hc.append(train_data_hc[i-60:i, 0])
  y_train_hc.append(train_data_hc[i, 0])

In [0]:
x_train_f, y_train_f = np.array(x_train_f), np.array(y_train_f)
x_train_hc, y_train_hc = np.array(x_train_hc), np.array(y_train_hc)

In [0]:
x_train_f = np.reshape(x_train_f, (x_train_f.shape[0], x_train_f.shape[1], 1))
x_train_hc = np.reshape(x_train_hc, (x_train_hc.shape[0], x_train_hc.shape[1], 1))

In [0]:
model_f = Sequential()
model_f.add(LSTM(50, return_sequences=True, input_shape = (x_train_f.shape[1], 1)))
model_f.add(LSTM(50, return_sequences=False))
model_f.add(Dense(25))
model_f.add(Dense(1))

In [0]:
model_hc = Sequential()
model_hc.add(LSTM(50, return_sequences=True, input_shape = (x_train_hc.shape[1], 1)))
model_hc.add(LSTM(50, return_sequences=False))
model_hc.add(Dense(25))
model_hc.add(Dense(1))

In [0]:
model_f.compile(optimizer='adam', loss='mean_squared_error')
model_hc.compile(optimizer='adam', loss='mean_squared_error')

In [0]:
model_f.fit(x_train_f, y_train_f, batch_size=1, epochs=2)

In [0]:
model_hc.fit(x_train_hc, y_train_hc, batch_size=1, epochs=2)

In [0]:
test_data_f = scaled_data_f[training_data_len-60:, :]
x_test_f = []
y_test_f = dataset_f[training_data_len:, :]
for i in range(60, len(test_data_f)):
  x_test_f.append(test_data_f[i-60:i, 0])

In [0]:
test_data_hc = scaled_data_hc[training_data_len-60:, :]
x_test_hc = []
y_test_hc = dataset_hc[training_data_len:, :]
for i in range(60, len(test_data_hc)):
  x_test_hc.append(test_data_hc[i-60:i, 0])

In [0]:
x_test_f = np.array(x_test_f)
x_test_f = np.reshape(x_test_f, (x_test_f.shape[0], x_test_f.shape[1], 1))
x_test_hc = np.array(x_test_hc)
x_test_hc = np.reshape(x_test_hc, (x_test_hc.shape[0], x_test_hc.shape[1], 1))

In [0]:
predictions_f = model.predict(x_test_f)
predictions_f = scaler_f.inverse_transform(predictions_f)

predictions_hc = model.predict(x_test_hc)
predictions_hc = scaler_hc.inverse_transform(predictions_hc)

In [0]:
rmse_f = np.sqrt(np.mean(predictions_f-y_test_f)**2)
rmse_hc = np.sqrt(np.mean(predictions_hc-y_test_hc)**2)
print(rmse_f)
print(rmse_hc)

In [0]:
train_f = data_f[:training_data_len]
valid_f = data_f[training_data_len:]
valid_f['Predictions'] = predictions_f

train_hc = data_hc[:training_data_len]
valid_hc = data_hc[training_data_len:]
valid_hc['Predictions'] = predictions_hc

In [0]:
plt.figure(figsize=(16,8))
plt.title('Model DJUSGF', fontsize = 18)
plt.xlabel('Date', fontsize = 14)
plt.ylabel('Close Price', fontsize = 14)
plt.plot(train_f['DJUSGF'])
plt.plot(valid_f[['DJUSGF', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'])
plt.figure(figsize=(16,8))
plt.title('Model DJUSHC', fontsize = 18)
plt.xlabel('Date', fontsize = 14)
plt.ylabel('Close Price', fontsize = 14)
plt.plot(train_hc['DJUSHC'])
plt.plot(valid_hc[['DJUSHC', 'Predictions']])
plt.legend(['Train', 'Val', 'Predictions'])
plt.show()

In [0]:
djf = valid_f['DJUSGF'].values.reshape(-1, 1).flatten().tolist()
pre_f = valid_f['Predictions'].values.reshape(-1, 1).flatten().tolist()
djhc = valid_hc['DJUSHC'].values.reshape(-1, 1).flatten().tolist()
pre_hc = valid_hc['Predictions'].values.reshape(-1, 1).flatten().tolist()

In [0]:
aic.aic(djf, pre_f, 60)

In [0]:
aic.aic(djhc, pre_hc, 60)